In [10]:
import plotly.offline as pyo
import plotly.graph_objs as go
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openai_api_request import make_openai_request
import pandas_ta as ta




In [11]:
ticker = 'SPY'
interval = '1h'
daily_data = yf.download(ticker,'2022-12-01','2023-01-24', interval=interval)

#if daily data has any nan values, drop the entire row

daily_data['hlc3'] = (daily_data['High'] + daily_data['Low'] + daily_data['Close']) / 3

[*********************100%***********************]  1 of 1 completed


In [12]:
def dthp(daily_data, lb_days, mode):

    '''Calculate the date time of highest price, lowest price and highest volume on a daily in the last lb_days days'''
    dthp_index = daily_data['High'][len(daily_data)-lb_days:].idxmax()
    index_hp = daily_data.index.get_loc(dthp_index)

    dtlp_index = daily_data['Low'][len(daily_data)-lb_days:].idxmin()
    index_lp = daily_data.index.get_loc(dtlp_index)

    dthv_index = daily_data['Volume'][len(daily_data)-lb_days:].idxmax()
    index_hv = daily_data.index.get_loc(dthv_index)

    if mode == "dates":
        return dthp_index, dtlp_index, dthv_index
        
    elif mode == "indexes":
        return index_hp, index_lp, index_hv


Change The (# Lookback Days) Arguement For The AAVWAP Indicator

In [13]:
lb_param = 1000

In [14]:
dthp_index, dtlp_index, dthv_index = dthp(daily_data, lb_param, "indexes")
dthp_date, dtlp_date, dthv_date = dthp(daily_data, lb_param, "dates")
print(dthp_index, dtlp_index, dthv_index)


54 107 54


In [15]:
hp_vwap_bars = daily_data[dthp_index:].copy()
lp_vwap_bars = daily_data[dtlp_index:].copy()
hv_vwap_bars = daily_data[dthv_index:].copy()

def vwap(vwap_bars):
    '''Calculate the volume weighted average price'''
    vwap_bars['vwap'] = (vwap_bars['hlc3'] * vwap_bars['Volume']).cumsum() / vwap_bars['Volume'].cumsum()
    return vwap_bars['vwap']

In [16]:
hpvwap = vwap(hp_vwap_bars)
lpvwap = vwap(lp_vwap_bars)
hvvwap = vwap(hv_vwap_bars)

#line up the vwap series and add it to daily_data
daily_data['hpvwap'] = hpvwap
daily_data['lpvwap'] = lpvwap
daily_data['hvvwap'] = hvvwap

In [17]:


trace = go.Candlestick(x=daily_data.index,
                        open=daily_data['Open'],
                        high=daily_data['High'],
                        low=daily_data['Low'],
                        close=daily_data['Close'],
                       )

high_vwap_plotlyobj = go.Scatter(x=daily_data.index, y=daily_data['hpvwap'], name='HPA: ' + str(dthp_date), line=dict(color='red'))
low_vwap_plotlyobj = go.Scatter(x=daily_data.index, y=daily_data['lpvwap'], name='LPA', line=dict(color='green'))
high_volume_vwap_plotlyobj = go.Scatter(x=daily_data.index, y=daily_data['hvvwap'], name='HVA', line=dict(color='blue'))
# rsi_obj = go.Scatter(x=daily_data.index, y=daily_data['RSI-14d'], name='RSI', line=dict(color='black'))
# rsi_upper_bound_obj = go.Scatter(x=daily_data.index, y=daily_data['rsi_upper_bound'], name='RSI Upper Bound', line=dict(color='black', dash='dash'))
# rsi_lower_bound_obj = go.Scatter(x=daily_data.index, y=daily_data['rsi_lower_bound'], name='RSI Lower Bound', line=dict(color='black', dash='dash'))

#plot a vertical line going down from the anchor candles.
vert = go.layout.Shape(type="line", x0=dthp_date, y0=0, x1=dthp_date, y1=daily_data["Low"][dthp_index], line=dict(color="red", width=2, dash='dash'), layer='below')
vert2 = go.layout.Shape(type="line", x0=dtlp_date, y0=0, x1=dtlp_date, y1=daily_data["Low"][dtlp_index], line=dict(color="green", width=2, dash='dash'), layer='below')
vert3 = go.layout.Shape(type="line", x0=dthv_date, y0=0, x1=dthv_date, y1=daily_data["Low"][dthv_index], line=dict(color="blue", width=2, dash='dash'), layer='below')

layout = go.Layout(shapes=[vert, vert2, vert3], title= 'Chart ' + ticker + ' ' + interval, xaxis=dict(title='Date'), yaxis=dict(title='Price'))
plotstuff = [trace, high_vwap_plotlyobj, low_vwap_plotlyobj, high_volume_vwap_plotlyobj, rsi_obj, rsi_upper_bound_obj, rsi_lower_bound_obj]

# layout = go.Layout(title= 'Chart ' + ticker + ' ' + interval, xaxis=dict(title='Date'), yaxis=dict(title='Price'))

fig = go.Figure(data=plotstuff, layout=layout)

#scale y axis to fit the data



NameError: name 'rsi_obj' is not defined

In [ ]:
fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                dict(count=6,
                        label="6m",
                        step="month",
                        stepmode="backward"),
                dict(count=1,
                        label="YTD",
                        step="year",
                        stepmode="todate"),
                dict(count=1,
                        label="1y",
                        step="year",
                        stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)



# fig.update_yaxes(range=[daily_data['Low'].min(), daily_data['High'].max()])
pyo.plot(fig, filename="AAVWAP.html")


NameError: name 'fig' is not defined